In [ ]:
### GUNAKAN Google Colab untuk menjalankan kode ini
# Deteksi Wajah dengan Viola-Jones dan Dlib
# Menggunakan OpenCV untuk deteksi wajah dan landmark, serta menghitung simetri wajah

# ======================================================================
# BAGIAN 1: INSTALASI DAN PERSIAPAN (DENGAN PENGECEKAN FILE)
# ======================================================================
import os

# Cek dan download file jika belum ada
if not os.path.exists('haarcascade_frontalface_default.xml'):
    !wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml -q
if not os.path.exists('shape_predictor_68_face_landmarks.dat'):
    !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -q
    !bzip2 -d shape_predictor_68_face_landmarks.dat.bz2 -q
if 'dlib' not in locals() and 'dlib' not in globals():
    !pip install dlib -q

print("✅ Persiapan Selesai. Semua file dan library siap digunakan.")

# ======================================================================
# BAGIAN 2: KODE UTAMA STREAMING REAL-TIME (DENGAN TOMBOL STOP)
# ======================================================================
from IPython.display import display, Javascript, Image
from google.colab.output import register_callback
import cv2
import dlib
import numpy as np
import base64

# Muat model detektor dan prediktor
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Fungsi untuk mengubah gambar OpenCV ke format yang bisa dikirim ke browser
def cv2_to_base64(img):
    _, bts = cv2.imencode('.jpeg', img)
    return f"data:image/jpeg;base64,{base64.b64encode(bts).decode('utf-8')}"

# Fungsi kalkulasi simetri
def calculate_symmetry(landmarks_points):
    SYMMETRY_PAIRS = [
        (0, 16), (1, 15), (2, 14), (3, 13), (4, 12), (5, 11), (6, 10), (7, 9),
        (17, 26), (18, 25), (19, 24), (20, 23), (21, 22),
        (36, 45), (37, 44), (38, 43), (39, 42),
        (48, 54), (49, 53), (50, 52), (60, 64), (61, 63)
    ]
    if not landmarks_points: return 0, 0
    center_x = np.mean([p.x for p in landmarks_points])
    total_asymmetry_score = sum(abs(abs(landmarks_points[li].x - center_x) - abs(landmarks_points[ri].x - center_x)) for li, ri in SYMMETRY_PAIRS)
    return total_asymmetry_score / len(SYMMETRY_PAIRS), center_x

# Fungsi yang akan dipanggil oleh JavaScript untuk setiap frame
def process_frame(data):
    binary = base64.b64decode(data.split(',')[1])
    nparr = np.frombuffer(binary, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    frame = cv2.flip(frame, 1) # Efek cermin

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 203), 2)
        dlib_rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
        landmarks = landmark_predictor(gray_frame, dlib_rect)

        landmark_points = [landmarks.part(n) for n in range(68)]
        for point in landmark_points:
            cv2.circle(frame, (point.x, point.y), 2, (135, 206, 250), -1)

        asymmetry_score, center_x = calculate_symmetry(landmark_points)
        cv2.line(frame, (int(center_x), y), (int(center_x), y+h), (255, 255, 0), 1)

        AMBANG_BATAS = 3.0
        final_label = "Ganteng" if asymmetry_score < AMBANG_BATAS else "Jelek"
        label_color = (0, 255, 0) if final_label == "Ganteng" else (0, 0, 255)

        cv2.putText(frame, final_label, (x, y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.9, label_color, 2)
        cv2.putText(frame, f"Skor Asimetri: {asymmetry_score:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    return cv2_to_base64(frame)

# Daftarkan fungsi Python agar bisa dipanggil oleh JavaScript
register_callback('processFrame', process_frame)

# Fungsi untuk memulai streaming video dari browser
def start_video_stream():
  js_code = Javascript('''
    var video;
    var stream;
    var image;
    var stopButton;
    var intervalId; // Variabel untuk menyimpan ID interval

    async function startVideo() {
      // Buat elemen-elemen HTML
      const div = document.createElement('div');
      image = document.createElement('img');
      image.style.display = 'block';
      image.style.marginBottom = '10px';

      // -- BAGIAN BARU: BUAT TOMBOL STOP --
      stopButton = document.createElement('button');
      stopButton.textContent = 'Hentikan Kamera';
      stopButton.style.padding = '10px 20px';
      stopButton.style.border = 'none';
      stopButton.style.borderRadius = '5px';
      stopButton.style.backgroundColor = '#dc3545';
      stopButton.style.color = 'white';
      stopButton.style.cursor = 'pointer';

      document.body.appendChild(div);
      div.appendChild(image);
      div.appendChild(stopButton);

      // Minta izin akses kamera
      stream = await navigator.mediaDevices.getUserMedia({video: true});
      video = document.createElement('video');
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // -- BAGIAN BARU: TAMBAHKAN FUNGSI ONCLICK PADA TOMBOL -- //
      stopButton.onclick = stopVideo;

      // -- BAGIAN BARU: SIMPAN ID INTERVAL AGAR BISA DIHENTIKAN -- //
      intervalId = setInterval(async () => {
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        let data = canvas.toDataURL('image/jpeg', 0.8);

        const result = await google.colab.kernel.invokeFunction('processFrame', [data], {});
        image.src = result.data['text/plain'].replace(/'/g, '');
      }, 100);
    }

    // -- BAGIAN BARU: FUNGSI UNTUK MENGHENTIKAN SEMUANYA --
    function stopVideo() {
      clearInterval(intervalId); // Hentikan loop pengiriman frame
      stream.getTracks().forEach(track => track.stop()); // Matikan kamera

      // Hapus semua elemen dari tampilan //
      const elements = document.body.getElementsByTagName('div');
      while (elements[0]) {
        elements[0].parentNode.removeChild(elements[0]);
      }
    }

    startVideo();
  ''')
  display(js_code)

# --- MULAI PROGRAM ---
print("Mempersiapkan streaming kamera...")
start_video_stream()

✅ Persiapan Selesai. Semua file dan library siap digunakan.
Mempersiapkan streaming kamera...


<IPython.core.display.Javascript object>

# **Penjelasan Teknis: Sistem Analisis Simetri Wajah Real-Time**

Berikut adalah penjabaran teknis mengenai arsitektur, alur kerja, dan algoritma yang digunakan untuk membangun sistem ini.

### **1. Arsitektur & Teknologi Inti**

Sistem ini dibangun di atas arsitektur *client-server*, di mana *client* adalah browser pengguna dan *server* adalah lingkungan eksekusi Google Colab. Komunikasi antara keduanya dijembatani oleh JavaScript.

Teknologi inti yang menjadi fondasi proyek ini adalah:

* **Python**: Sebagai bahasa pemrograman utama di sisi *backend* untuk semua logika pemrosesan.
* **OpenCV (`cv2`)**: Library yang digunakan untuk dua tugas utama:
    1.  **Pemrosesan Gambar Dasar**: Membaca, mengubah format warna, membalik gambar, dan menggambar bentuk (kotak, teks).
    2.  **Deteksi Wajah**: Menjalankan algoritma **Viola-Jones** melalui `cv2.CascadeClassifier`.
* **Dlib**: Library yang sangat spesifik perannya, yaitu untuk **Deteksi Facial Landmark**. Setelah wajah ditemukan oleh OpenCV, Dlib mengambil alih untuk memetakan 68 titik kunci di wajah dengan presisi tinggi.
* **JavaScript**: Berperan sebagai "jembatan" krusial di sisi *client* (browser). Fungsinya adalah mengakses perangkat keras lokal (kamera) dan mengelola komunikasi *real-time* dengan *backend* Python di Colab.

---

### **2. Alur Kerja Teknis per Frame**

Untuk setiap frame yang ditangkap dari kamera, sistem melakukan serangkaian langkah berikut secara berurutan dan sangat cepat (sekitar 10 kali per detik):

1.  **Akses Kamera & Pengambilan Frame (JavaScript)**: Fungsi `navigator.mediaDevices.getUserMedia` di JavaScript meminta izin dan mengaktifkan kamera web pengguna.
2.  **Encoding Data (JavaScript)**: Frame video yang tertangkap diubah menjadi gambar format JPEG, lalu di-*encode* menjadi format teks **Base64**. Ini dilakukan agar data gambar bisa dikirim melalui jaringan sebagai teks biasa.
3.  **Pengiriman ke Python (JavaScript -> Python)**: JavaScript memanggil fungsi Python yang telah didaftarkan (`processFrame`) menggunakan `google.colab.kernel.invokeFunction`, dengan data Base64 sebagai argumennya.
4.  **Penerimaan & Decoding (Python)**: Fungsi `process_frame` di Python menerima data Base64, men-*decode*-nya kembali menjadi data biner, lalu `cv2.imdecode` mengubahnya menjadi gambar format OpenCV (Array NumPy) yang siap diolah.
5.  **Deteksi Wajah - Algoritma Viola-Jones (OpenCV)**: Gambar diubah ke *grayscale*. Fungsi `face_detector.detectMultiScale()` dijalankan. Ini adalah implementasi dari algoritma **Viola-Jones** yang secara cepat memindai gambar untuk menemukan area yang memiliki ciri-ciri seperti wajah. Outputnya adalah koordinat kotak pembatas `(x, y, w, h)`.
6.  **Deteksi Landmark - Model Dlib (Dlib)**: Untuk setiap wajah yang ditemukan, area di dalam kotak pembatas diserahkan ke `landmark_predictor`. Ini adalah model *machine learning* dari Dlib yang telah dilatih untuk secara akurat memetakan posisi **68 titik spesifik** (sudut mata, kontur bibir, dll.) di wajah.
7.  **Kalkulasi Skor Asimetri (Logika Python)**: Ke-68 titik landmark ini kemudian dimasukkan ke dalam fungsi `calculate_symmetry` untuk dihitung skor asimetrinya. Proses detail dijelaskan di bagian selanjutnya.
8.  **Klasifikasi & Visualisasi (Python/OpenCV)**: Berdasarkan skor yang dihasilkan, sebuah label ("Simetris" / "Tidak Simetris") ditentukan. Label ini, beserta kotak wajah, titik-titik landmark, dan garis tengah, digambar di atas frame gambar menggunakan fungsi seperti `cv2.putText` dan `cv2.rectangle`.
9.  **Pengiriman Balik ke Browser (Python -> JavaScript)**: Gambar yang sudah dianotasi di-*encode* kembali menjadi format Base64 dan dikirim sebagai nilai *return* dari fungsi `processFrame`.
10. **Tampilan Hasil (JavaScript)**: JavaScript menerima data gambar hasil olahan ini dan secara dinamis memperbarui sumber (`src`) dari elemen `<img>` di halaman, sehingga pengguna melihat output yang diperbarui.

---

### **3. Bedah Tuntas: Proses Perhitungan Asimetri dan Klasifikasi**

Bagian ini adalah inti dari logika analisis proyek. Prosesnya murni **pengukuran geometris** yang diikuti oleh **klasifikasi berbasis aturan (rule-based)**.

Berikut adalah 5 langkah detail dari input (68 titik wajah) hingga output (label klasifikasi):

* **Langkah 1: Menentukan Garis Tengah Wajah (Sumbu Referensi)**
    Langkah pertama adalah menentukan sebuah garis referensi vertikal. Garis ini dianggap sebagai sumbu simetri wajah. Cara paling sederhana untuk mendapatkannya adalah dengan menghitung nilai rata-rata dari seluruh koordinat horizontal (sumbu-x) dari ke-68 titik landmark.
    * **Logika Kode**: `center_x = np.mean([p.x for p in landmarks_points])`

* **Langkah 2: Mendefinisikan Pasangan Titik Simetris**
    Program tidak membandingkan semua titik, melainkan hanya titik-titik yang secara anatomis memiliki pasangan simetris. Pasangan ini didefinisikan secara eksplisit di dalam sebuah list bernama `SYMMETRY_PAIRS`.
    * **Contoh Pasangan**: `(36, 45)` yang merepresentasikan sudut luar mata kiri dan kanan, atau `(48, 54)` yang merepresentasikan sudut bibir kiri dan kanan.

* **Langkah 3: Menghitung "Error" Simetri untuk Setiap Pasangan**
    Untuk setiap pasangan titik, program mengukur seberapa "tidak simetris" mereka terhadap garis tengah yang sudah ditentukan pada Langkah 1.
    1.  Ukur jarak horizontal dari **titik kiri** ke **garis tengah**.
    2.  Ukur jarak horizontal dari **titik kanan** ke **garis tengah**.
    3.  Hitung selisih absolut dari kedua jarak tersebut: `selisih = abs(jarak_kiri - jarak_kanan)`.
    Nilai `selisih` ini merepresentasikan "error" atau tingkat ketidaksimetrisan untuk satu pasang titik tersebut. Jika sempurna simetris, nilainya akan 0.

* **Langkah 4: Akumulasi dan Normalisasi Skor**
    Semua nilai "error" dari setiap pasangan titik tadi dijumlahkan untuk mendapatkan skor total asimetri. Untuk membuat skor ini lebih stabil dan mudah diinterpretasikan, skor total tersebut kemudian dibagi dengan jumlah pasangan yang diuji.
    * **Output**: Hasil akhir dari langkah ini adalah sebuah angka tunggal yang merepresentasikan **rata-rata error asimetri**. Wajah yang sangat simetris akan memiliki skor mendekati 0, dan semakin tidak simetris sebuah wajah, semakin tinggi skornya.

* **Langkah 5: Klasifikasi Berbasis Ambang Batas (Thresholding)**
    Di sinilah keputusan akhir dibuat. Skor rata-rata error asimetri dari Langkah 4 dibandingkan dengan sebuah nilai yang telah kita tetapkan, yaitu `AMBANG_BATAS`.
    * **Definisi**: `AMBANG_BATAS` (misalnya, diatur ke `2.0`) adalah nilai toleransi kita terhadap ketidaksimetrisan. Nilai ini bersifat **empiris**, artinya didapatkan dari hasil eksperimen dan pengamatan untuk menemukan angka yang paling masuk akal.
    * **Logika Keputusan**:
        * `if asymmetry_score < AMBANG_BATAS:`: Jika rata-rata error lebih KECIL dari toleransi kita, maka sistem menyimpulkan bahwa wajah tersebut cukup simetris. **Label: "Simetris"**.
        * `else:`: Jika rata-rata error lebih BESAR dari atau sama dengan toleransi kita, maka sistem menyimpulkan bahwa tingkat ketidaksimetrisannya signifikan. **Label: "Tidak Simetris"**.

---

### **4. Mekanisme Streaming Real-Time di Colab**

Mekanisme ini adalah solusi untuk tantangan utama, yaitu menghubungkan kamera lokal ke server cloud.

* **Peran JavaScript (Di Browser Anda)**:
    1.  `navigator.mediaDevices.getUserMedia`: Mengaktifkan kamera.
    2.  `setInterval(..., 100)`: Membuat sebuah *timer* yang akan menjalankan sebuah fungsi setiap 100 milidetik.
    3.  `google.colab.kernel.invokeFunction(...)`: Di dalam *timer* tersebut, fungsi ini dipanggil untuk "mengetuk pintu" Python di server Colab dan mengirimkan data frame.

* **Peran Python (Di Server Colab)**:
    1.  `register_callback('processFrame', ...)`: Mendaftarkan fungsi `process_frame` dan membuatnya "mendengarkan" panggilan dari JavaScript dengan nama `processFrame`.
    2.  **Pemrosesan**: Saat "ketukan pintu" diterima, `process_frame` berjalan, melakukan semua analisis (Viola-Jones, Dlib, kalkulasi).
    3.  **Mengirim Balasan**: Setelah selesai, `process_frame` mengembalikan gambar hasil anotasi. JavaScript yang tadinya memanggil akan menerima balasan ini dan menampilkannya di layar.

Siklus **Panggil-Proses-Balas** inilah yang terjadi berulang kali dengan sangat cepat, sehingga menciptakan ilusi video *real-time* yang interaktif.